In [ ]:
pip install pandas numpy mlxtend lightgbm hyperopt shap matplotlib seaborn lime pdpbox pyALE

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, KFold, cross_validate
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.svm import SVR
from sklearn.ensemble import StackingRegressor, RandomForestRegressor as RFR
from sklearn.neural_network import MLPRegressor
import lightgbm as lgb
from hyperopt import fmin, tpe, hp, Trials
import warnings
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error, explained_variance_score
import shap
from sklearn.linear_model import HuberRegressor

In [ ]:
warnings.filterwarnings("ignore")

data = pd.read_excel(r'C:\Users\28901\Desktop\水生生物\新的分子描述符\17个分子描述符鱼类带Y.xlsx', index_col=0)

X = data.iloc[:, 1:-1]  # 特征列
Y = data.iloc[:, -1]  # 目标列

Xtrain, Xtest, Ytrain, Ytest = train_test_split(X, Y, test_size=0.3, random_state=100)

scaler_X = MinMaxScaler()
Xtrain_scaled = scaler_X.fit_transform(Xtrain)
Xtest_scaled = scaler_X.transform(Xtest)

clf1 = LinearRegression()
clf2 = Ridge(alpha=1.0)
clf3 = SVR(C=1.0, epsilon=0.2)
clf4 = lgb.LGBMRegressor(n_estimators=100, learning_rate=0.05, num_leaves=31) 
clf5 = MLPRegressor(hidden_layer_sizes=(100,), max_iter=1000, random_state=100)

meta_regressor = RFR(n_estimators=100, max_depth=5, random_state=100)

def hyperopt_objective_LGB(params):
    clf4 = lgb.LGBMRegressor(
        n_estimators=int(params["n_estimators"]),
        learning_rate=params["learning_rate"],
        num_leaves=int(params["num_leaves"])
    )
    stack = StackingRegressor(estimators=[('clf1', clf1), ('clf2', clf2), ('clf3', clf3), 
                                          ('clf4', clf4), ('clf5', clf5)], final_estimator=meta_regressor)

    cv = KFold(n_splits=10, shuffle=True, random_state=100)
    validation_loss = cross_validate(stack, Xtrain_scaled, Ytrain, scoring="r2", cv=cv, n_jobs=-1)
    return -np.mean(validation_loss["test_score"])

def hyperopt_objective_MLP(params):
    clf5 = MLPRegressor(
        hidden_layer_sizes=(int(params["hidden_layer_size"]),),
        max_iter=1000,
        random_state=100
    )
    stack = StackingRegressor(estimators=[('clf1', clf1), ('clf2', clf2), ('clf3', clf3), 
                                          ('clf4', clf4), ('clf5', clf5)], final_estimator=meta_regressor)

    cv = KFold(n_splits=10, shuffle=True, random_state=100)
    validation_loss = cross_validate(stack, Xtrain_scaled, Ytrain, scoring="r2", cv=cv, n_jobs=-1)
    return -np.mean(validation_loss["test_score"])

def hyperopt_objective_Ridge(params):
    clf2 = Ridge(alpha=params["alpha"])
    stack = StackingRegressor(estimators=[('clf1', clf1), ('clf2', clf2), ('clf3', clf3), 
                                          ('clf4', clf4), ('clf5', clf5)], final_estimator=meta_regressor)

    cv = KFold(n_splits=10, shuffle=True, random_state=100)
    validation_loss = cross_validate(stack, Xtrain_scaled, Ytrain, scoring="r2", cv=cv, n_jobs=-1)
    return -np.mean(validation_loss["test_score"])

def hyperopt_objective_SVR(params):
    clf3 = SVR(C=params["C"], epsilon=params["epsilon"])
    stack = StackingRegressor(estimators=[('clf1', clf1), ('clf2', clf2), ('clf3', clf3), 
                                          ('clf4', clf4), ('clf5', clf5)], final_estimator=meta_regressor)

    cv = KFold(n_splits=10, shuffle=True, random_state=100)
    validation_loss = cross_validate(stack, Xtrain_scaled, Ytrain, scoring="r2", cv=cv, n_jobs=-1)
    return -np.mean(validation_loss["test_score"])

def hyperopt_objective_RFR(params):
    meta_regressor = RFR(n_estimators=int(params["n_estimators"]), 
                         max_depth=int(params["max_depth"]), 
                         min_samples_split=int(params["min_samples_split"]),
                         random_state=100)
    
    stack = StackingRegressor(estimators=[('clf1', clf1), ('clf2', clf2), ('clf3', clf3), 
                                          ('clf4', clf4), ('clf5', clf5)], final_estimator=meta_regressor)

    cv = KFold(n_splits=10, shuffle=True, random_state=100)
    validation_loss = cross_validate(stack, Xtrain_scaled, Ytrain, scoring="r2", cv=cv, n_jobs=-1)
    return -np.mean(validation_loss["test_score"])
    
param_space_LGB = {
    'n_estimators': hp.quniform('n_estimators', 50, 200, 10),
    'learning_rate': hp.uniform('learning_rate', 0.01, 0.1),
    'num_leaves': hp.quniform('num_leaves', 20, 100, 10),
}

param_space_MLP = {
    'hidden_layer_size': hp.quniform('hidden_layer_size', 50, 200, 10),
}

param_space_Ridge = {
    'alpha': hp.uniform('alpha', 0.1, 10),
}

param_space_SVR = {
    'C': hp.uniform('C', 0.1, 10),
    'epsilon': hp.uniform('epsilon', 0.01, 0.1),
}

param_space_RFR = {
    'n_estimators': hp.quniform('n_estimators', 50, 200, 10),
    'max_depth': hp.quniform('max_depth', 5, 20, 1),
    'min_samples_split': hp.quniform('min_samples_split', 2, 10, 1),
}

def optimize_hyperparameters_LGB(param_space):
    trials = Trials()
    best_params = fmin(fn=hyperopt_objective_LGB, space=param_space, algo=tpe.suggest, max_evals=50, trials=trials)
    print(f"Best Parameters for LGB:", best_params)
    return best_params

def optimize_hyperparameters_MLP(param_space):
    trials = Trials()
    best_params = fmin(fn=hyperopt_objective_MLP, space=param_space, algo=tpe.suggest, max_evals=50, trials=trials)
    print(f"Best Parameters for MLP:", best_params)
    return best_params

def optimize_hyperparameters_Ridge(param_space):
    trials = Trials()
    best_params = fmin(fn=hyperopt_objective_Ridge, space=param_space, algo=tpe.suggest, max_evals=50, trials=trials)
    print(f"Best Parameters for Ridge:", best_params)
    return best_params

def optimize_hyperparameters_SVR(param_space):
    trials = Trials()
    best_params = fmin(fn=hyperopt_objective_SVR, space=param_space, algo=tpe.suggest, max_evals=50, trials=trials)
    print(f"Best Parameters for SVR:", best_params)
    return best_params

def optimize_hyperparameters_RFR(param_space):
    trials = Trials()
    best_params = fmin(fn=hyperopt_objective_RFR, space=param_space, algo=tpe.suggest, max_evals=50, trials=trials)
    print(f"Best Parameters for RFR:", best_params)
    return best_params

best_params_LGB = optimize_hyperparameters_LGB(param_space_LGB)
best_params_MLP = optimize_hyperparameters_MLP(param_space_MLP)
best_params_Ridge = optimize_hyperparameters_Ridge(param_space_Ridge)
best_params_SVR = optimize_hyperparameters_SVR(param_space_SVR)
best_params_RFR = optimize_hyperparameters_RFR(param_space_RFR)

optimized_clf4 = lgb.LGBMRegressor(
    n_estimators=int(best_params_LGB['n_estimators']),
    learning_rate=best_params_LGB['learning_rate'],
    num_leaves=int(best_params_LGB['num_leaves'])
)

optimized_clf5 = MLPRegressor(
    hidden_layer_sizes=(int(best_params_MLP['hidden_layer_size']),),
    max_iter=1000,
    random_state=100
)

optimized_clf2 = Ridge(alpha=best_params_Ridge['alpha'])

optimized_clf3 = SVR(C=best_params_SVR['C'], epsilon=best_params_SVR['epsilon'])

meta_regressor = RFR(n_estimators=int(best_params_RFR['n_estimators']),
                     max_depth=int(best_params_RFR['max_depth']),
                     min_samples_split=int(best_params_RFR['min_samples_split']),
                     random_state=100)

stack_optimized = StackingRegressor(
    estimators=[('clf1', clf1), ('clf2', optimized_clf2), ('clf3', optimized_clf3), 
                ('clf4', optimized_clf4), ('clf5', optimized_clf5)],
    final_estimator=meta_regressor
)

cv = KFold(n_splits=10, shuffle=True, random_state=100)
validation_loss = cross_validate(stack_optimized, Xtrain_scaled, Ytrain, scoring="r2", cv=cv, n_jobs=-1)
score_cv = np.mean(validation_loss["test_score"])

stack_optimized.fit(Xtrain_scaled, Ytrain)

y_pred = stack_optimized.predict(Xtest_scaled)

In [ ]:
def huber_loss(y_true, y_pred, delta=1.35):
    error = y_true - y_pred
    loss = np.where(
        np.abs(error) <= delta,
        0.5 * error ** 2,  # 对于小误差部分
        delta * (np.abs(error) - 0.5 * delta)  # 对于大误差部分
    )
    return np.mean(loss)

huber_loss_value = huber_loss(Ytest, y_pred)

print("Huber loss:", huber_loss_value)

In [ ]:
r2 = r2_score(Ytest, y_pred)
rmse_score = np.sqrt(mse_score)
print(f'堆叠回归器 R²（测试集）：{r2}')
print(f'堆叠回归器 RMSE：{rmse_score}')

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score

r2_test = r2_score(Ytest, y_pred)

z = np.polyfit(Ytest, y_pred, 1)
p = np.poly1d(z)

x_min, x_max = np.min(Ytest) - 0.5, np.max(Ytest) + 0.5
x_extended = np.linspace(x_min, x_max, 500) 
y_fit_extended = p(x_extended)

scale_factor = 1.5 
center_x = (x_max + x_min) / 2 
relative_width = np.abs(x_extended - center_x) / (x_max - x_min) 
std_residuals = np.std(y_pred - p(Ytest)) 
ci_upper = y_fit_extended + scale_factor * std_residuals * (1 + relative_width)  
ci_lower = y_fit_extended - scale_factor * std_residuals * (1 + relative_width) 

plt.rcParams['font.family'] = 'Times New Roman'
plt.rcParams["font.weight"] = "bold"

plt.figure(figsize=(8, 6), dpi=1200)

plt.scatter(Ytest, y_pred, color='coral', label="Predicted values", alpha=0.6)

plt.plot(x_extended, x_extended, color='grey', linestyle='--', label="1:1 Line", linewidth=1.5)

plt.plot(x_extended, y_fit_extended, color='blue', label=f"Line of Best Fit", linewidth=2)

plt.fill_between(x_extended, ci_lower, ci_upper, color='blue', alpha=0.2, label="95% Confidence Interval")

plt.title("Model 3", fontsize=14, fontname="Times New Roman")
plt.xlabel("Actual Values", fontsize=12, fontname="Times New Roman")
plt.ylabel("Predicted values", fontsize=12, fontname="Times New Roman")

plt.legend(loc="upper left", fontsize=10)

ax = plt.gca()
ax.spines['top'].set_linewidth(1.5)
ax.spines['right'].set_linewidth(1.5)
ax.spines['left'].set_linewidth(1.5)
ax.spines['bottom'].set_linewidth(1.5)

ax.xaxis.set_ticks_position('bottom')
ax.yaxis.set_ticks_position('left')
ax.tick_params(axis='x', which='both', direction='out', width=1.5)
ax.tick_params(axis='y', which='both', direction='out', width=1.5)

plt.savefig(r'C:\Users\28901\Desktop\水生生物\图\鱼类毒性拟合图new1.24.png', format='png',dpi=600, bbox_inches='tight')
plt.show()

In [ ]:
new_data = pd.read_excel(r'C:\Users\1\Desktop\水生生物\新的分子描述符\51个分子描述符预测.xlsx', index_col=0)
X_new = new_data.iloc[:, 1:]
X_new_scaled = scaler_X.transform(X_new)
y_new_pred = stack_optimized.predict(X_new_scaled)
new_data['Predicted_Y'] = y_new_pred
new_data.to_excel(r'C:\Users\1\Desktop\水生生物\新的分子描述符\51个分子描述符预测鱼类毒性12.20.xlsx', index=False)

In [ ]:
import seaborn as sns
from sklearn.inspection import permutation_importance

result = permutation_importance(stack_optimized, Xtest_scaled, Ytest, n_repeats=10, random_state=42)
importance = result.importances_mean
sorted_idx = importance.argsort()[::-1] 
sorted_importance = importance[sorted_idx] 
sorted_features = X.columns[sorted_idx] 
plt.rcParams["font.family"] = "Times New Roman"
plt.rcParams["font.weight"] = "bold"
plt.figure(figsize=(10, 8)) 
sns.barplot(x=sorted_importance, y=sorted_features, color="#24d7d0")
plt.title(r'$\it{Danio\ rerio}$ toxicity Feature Importance', fontweight='bold', fontsize=16)
plt.xlabel('Importance', fontsize=12, fontweight='bold')
plt.ylabel('Features', fontsize=12, fontweight='bold')
plt.xlim(0, max(sorted_importance) * 1.1) 
plt.gca().invert_yaxis() 
plt.tight_layout() 
for spine in plt.gca().spines.values():
    spine.set_linewidth(2) 
save_path = r'C:\Users\1\Desktop\水生生物\shap\特征重要性鱼类毒性.png' 
plt.savefig(save_path, format='png', dpi=600) 
plt.show()

In [ ]:
from sklearn.inspection import PartialDependenceDisplay

plt.rcParams["font.family"] = "Times New Roman"
plt.rcParams["font.weight"] = "bold"
features = ['Chi1', 'NumRotatableBonds', 'HeavyAtomCount']
for feature in features:
    feature_index = X.columns.get_loc(feature)
    fig, ax = plt.subplots(figsize=(8, 6))  
    disp = PartialDependenceDisplay.from_estimator(
        stack_optimized,             
        Xtrain_scaled,             
        features=[feature_index],   
        feature_names=X.columns,    
        grid_resolution=50,         
        ax=ax                     
    )
    for ax in disp.axes_.ravel():
        ax.set_title('')  
    for i, ax in enumerate(disp.axes_.ravel()):
        ax.set_xlabel(feature, fontsize=12, fontweight='bold')

    for ax in disp.axes_.ravel():
        for line in ax.get_lines():
            line.set_color("#9900FA") 
        for collection in ax.collections: 
            collection.set_facecolor("#9900FA")
    for ax in disp.axes_.ravel():
        for spine in ax.spines.values():
            spine.set_linewidth(2) 
    plt.subplots_adjust(hspace=1.0, top=0.95)
    save_path = f'C:/Users/1/Desktop/水生生物/依赖图/鱼类/{feature}_PDP.png' 
    plt.savefig(save_path, format='png', dpi=600, bbox_inches='tight')
    plt.tight_layout()
    plt.show()
    print(f"Partial dependence plot for {feature} saved at {save_path}")

In [ ]:
import shap
import os

plt.rcParams["font.family"] = "Times New Roman"
plt.rcParams["font.weight"] = "bold"
background = np.vstack([Xtrain_scaled, Xtest_scaled])
explainer = shap.KernelExplainer(stack_optimized.predict, background)
shap_values = explainer.shap_values(Xtest_scaled)
save_dir = r'C:\Users\1\Desktop\水生生物\shap'
os.makedirs(save_dir, exist_ok=True)
summary_plot_path = os.path.join(save_dir, "SHAP_summary_plot_fish12.20.png")
plt.figure(figsize=(10, 8))
shap.summary_plot(shap_values, Xtest_scaled, feature_names=X.columns, show=False)
plt.xlabel('Feature Importance', fontsize=14, weight='bold')
plt.ylabel('Features', fontsize=14, weight='bold')
plt.title('SHAP Summary Plot', fontsize=16, weight='bold')
for spine in plt.gca().spines.values():
    spine.set_linewidth(2)
plt.savefig(summary_plot_path, format="png", bbox_inches="tight", dpi=600)
plt.close()
bar_plot_path = os.path.join(save_dir, "SHAP_bar_plot_fish12.20.png")
plt.figure(figsize=(10, 8))
shap.summary_plot(shap_values, Xtest_scaled, plot_type="bar", feature_names=X.columns, show=False)
plt.xlabel('Feature Importance', fontsize=14, weight='bold')
plt.ylabel('Features', fontsize=14, weight='bold')
plt.title('SHAP Bar Plot', fontsize=16, weight='bold')
for spine in plt.gca().spines.values():
    spine.set_linewidth(2)
plt.savefig(bar_plot_path, format="png", bbox_inches="tight", dpi=600)
plt.close()